In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.8/330.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e063c2c853c38e9c0a2f4101764b81cea3882e0e705c57994944b9c08729c9b0
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


# Load pre_trained model (BERT)

In [2]:
import json
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model_name='bert-base-cased'
model_type='bert'

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
with open('/content/train-v1.1.json', 'r') as f:
    train_data = json.load(f)['data']

with open('/content/dev-v1.1.json', 'r') as f:
    eval_data = json.load(f)['data']

In [4]:
print("Train data structure:")
print(type(train_data))
if len(train_data) > 0:
    print("First item keys:", train_data[0].keys())
    print("First item:", train_data[0])

print("\nEval data structure:")
print(type(eval_data))
if len(eval_data) > 0:
    print("First item keys:", eval_data[0].keys())
    print("First item:", eval_data[0])

Train data structure:
<class 'list'>
First item keys: dict_keys(['title', 'paragraphs'])
First item: {'title': 'University_of_Notre_Dame', 'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'qas': [{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly 

In [5]:
model_args=QuestionAnsweringArgs()
model_args.train_batch_size=16
model_args.evaluate_during_training=True
model_args.num_train_epochs=3
model_args.n_best_size=3
model_args.save_model_every_epoch=False

In [6]:
train_args={
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 2,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save model every epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu":
}

In [7]:
model=QuestionAnsweringModel(model_type,model_name,args=train_args)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:

def flatten_data(data):
    flattened = []
    for item in data:
        for paragraph in item['paragraphs']:
            flattened.append(paragraph)
    return flattened

train_data_flattened = flatten_data(train_data)
eval_data_flattened = flatten_data(eval_data)
# Use a smaller subset for initial testing
train_data_small = train_data_flattened[:1000]
eval_data_small = eval_data_flattened[:200]
model.train_model(train_data_small, eval_data=eval_data_small)

model.train_model(train_data_flattened, eval_data=eval_data_flattened)

add example index and unique id: 100%|██████████| 5825/5825 [00:00<00:00, 1289210.11it/s]


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

(22116,
 {'global_step': [1000,
   2000,
   3000,
   4000,
   5000,
   6000,
   7000,
   8000,
   9000,
   10000,
   11000,
   11058,
   12000,
   13000,
   14000,
   15000,
   16000,
   17000,
   18000,
   19000,
   20000,
   21000,
   22000,
   22116],
  'correct': [4299,
   4464,
   4553,
   4563,
   4702,
   4665,
   4743,
   4686,
   4819,
   4826,
   4876,
   4884,
   4895,
   4908,
   4898,
   4986,
   4882,
   4893,
   4924,
   4908,
   4953,
   4972,
   4953,
   4953],
  'similar': [4613,
   4150,
   4597,
   4853,
   4588,
   4809,
   4768,
   4890,
   4746,
   4733,
   4770,
   4661,
   4560,
   4521,
   4698,
   4418,
   4770,
   4725,
   4663,
   4657,
   4632,
   4608,
   4616,
   4617],
  'incorrect': [1658,
   1956,
   1420,
   1154,
   1280,
   1096,
   1059,
   994,
   1005,
   1011,
   924,
   1025,
   1115,
   1141,
   974,
   1166,
   918,
   952,
   983,
   1005,
   985,
   990,
   1001,
   1000],
  'train_loss': [1.440114974975586,
   1.3620147705078125,
   0.717

In [10]:
result,texts=model.eval_model(eval_data_flattened)

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1194: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [12]:
to_predict=[{
    "context":"The capital of France is Paris.",
    "qas":[
        {
            "question":"Where is the capital of France?",
            "id":"0"
        }
    ]

}]

In [13]:
answers, probabilities=model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3182.32it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['Paris', 'Paris.', 'The capital of France is Paris']}]


/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1358: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
